In [73]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone
from dotenv import load_dotenv

In [74]:
load_dotenv()

True

In [92]:
from datasets import load_dataset

data = load_dataset("wikipedia", "20220301.simple", split='train[:10000]')
data

Downloading: 100%|██████████| 1.66k/1.66k [00:00<00:00, 7.34MB/s]
Downloading: 100%|██████████| 235M/235M [00:10<00:00, 22.4MB/s] 


Dataset wikipedia downloaded and prepared to /Users/isaac/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 10000
})

In [75]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

In [76]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

In [77]:
res = embeddings.embed_documents(texts)

In [78]:
res, len(res[0])

([[-0.011914792470633984,
   0.08918040990829468,
   0.03824290260672569,
   0.01908121444284916,
   0.05977566912770271,
   0.00531640974804759,
   0.03878142684698105,
   -0.008799010887742043,
   0.060234811156988144,
   -0.015470295213162899,
   0.027178583666682243,
   0.05887814983725548,
   -0.025384066626429558,
   -0.03383245691657066,
   -0.019123582169413567,
   -0.00024141022004187107,
   0.066612109541893,
   -0.061403416097164154,
   -0.004256842657923698,
   -0.0020314771682024,
   0.02755114436149597,
   0.1366371512413025,
   0.0045463647693395615,
   0.02143855020403862,
   0.04543561115860939,
   0.07764159142971039,
   -0.09471821784973145,
   0.06313018500804901,
   0.062286946922540665,
   -0.035114847123622894,
   -0.00600004568696022,
   -0.029994625598192215,
   0.1558808535337448,
   0.05987437814474106,
   0.016393404453992844,
   0.04131561890244484,
   0.0082614840939641,
   0.028696322813630104,
   0.02570163458585739,
   0.05118107795715332,
   0.03644582

In [79]:

# Save it into pinecone
API_KEY = os.environ.get("PINECONE_API_KEY")
YOUR_ENV = os.environ.get("PINECONE_ENVIRONMENT", "us-west4-gcp-free")
index_name = "test-abinvev"

In [80]:
pinecone.init(
    api_key=API_KEY,
    environment=YOUR_ENV
)

pinecone.list_indexes()

['test-abinvev']

In [87]:
pinecone.describe_index(index_name)

IndexDescription(name='test-abinvev', metric='cosine', replicas=1, dimension=1024.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [88]:
from tqdm.auto import tqdm
from uuid import uuid4

In [89]:
batch_limit = 100

texts = []
metadatas = []

In [93]:
for i, record in enumerate(tqdm(data)):
    # first get metadata fields for this record
    metadata = {
        'wiki-id': str(record['id']),
        'source': record['url'],
        'title': record['title']
    }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record['text'])
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

if len(texts) > 0:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))

  0%|          | 0/10000 [00:00<?, ?it/s]


NameError: name 'text_splitter' is not defined